In [19]:
import os
import pickle

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from starfish import FieldOfView, data
from starfish.types import Axes
from skimage import io

%matplotlib inline
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Load Data into Starfish from the Cloud

In [20]:
# Download the ISS data
experiment = data.ISS(use_test_data=False)

# Download the spots data as an ImageStack object
fov = experiment.fov()
primary_image = fov.get_image(FieldOfView.PRIMARY_IMAGES)

# Display the shape of the ImageStack
# The image data are stored in an x array
primary_image.xarray.shape

100%|██████████| 16/16 [00:00<00:00, 21.72it/s]


(4, 4, 1, 1044, 1390)

# Filter Data

In [21]:
from starfish.image import Filter

filter_ghp = Filter.GaussianHighPass(sigma=(1, 8, 8), is_volume=True)
imgs_ghp = filter_ghp.run(primary_image, in_place=False)

16it [00:00, 57.91it/s]


In [22]:
from starfish.types import Clip
filter_laplace = Filter.Laplace(sigma=(0.2, 0.5, 0.5), is_volume=True, clip_method=Clip.SCALE_BY_CHUNK)
imgs_ghp_laplace = filter_laplace.run(imgs_ghp, in_place=False)

16it [00:00, 41.38it/s]


# Save Images

In [23]:
mp = imgs_ghp_laplace.max_proj(Axes.ZPLANE)

100%|██████████| 16/16 [00:00<00:00, 103.97it/s]


In [24]:
# %gui qt5
# from starfish import display
# display(mp)

In [25]:
# display(imgs_ghp)

In [26]:
# Get the number of channels in the ImageStack
num_ch = mp.num_chs

# Get the number of round in the ImageStack
num_rnd = mp.num_rounds

# Get the number of z slices in the ImageStack
num_z = mp.num_zplanes

print('%d rounds, %d channels, and %d z slice(s)' % (num_rnd, num_ch, num_z))

4 rounds, 4 channels, and 1 z slice(s)


In [27]:
if not os.path.exists('images'):
    os.mkdir('images')
# Iterate through the images and save them to disk
for rnd in range(num_rnd):
    for channel in range(num_ch):
        for z in range(num_z):
            # Create the file name
            file_name = 'ISS_rnd%d_ch%d_z%d.png' % (rnd, channel, z)
            file_path = './images/%s' % file_name
            
            # Create the selector. This dictionary selects which slice (i.e., round, channel, z coords)
            # to select from the image stack
            selector = {
                Axes.ROUND: rnd,
                Axes.CH: channel,
                Axes.ZPLANE: z
            }
            
            # Get the slice
            (im, _) = mp.get_slice(selector)
            
            # Save the slice
            io.imsave(file_path, im, check_contrast=False)


/usr/local/lib/python3.6/site-packages/skimage/util/dtype.py:130: UserWarning: Possible precision loss when converting from float32 to uint16
  .format(dtypeobj_in, dtypeobj_out))
